In [383]:
import pandas as pd



In [384]:
dataset=pd.read_csv(r"C:\Users\wwrao\Documents\machine project without deployment\diabetes pediction\Prediction using decison tree\diabetes_prediction_dataset.csv")

In [385]:
dataset.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [386]:
dataset.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [387]:
dataset.describe()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [388]:
dataset.shape

(100000, 9)

In [389]:
dataset["smoking_history"].value_counts()

smoking_history
No Info        35816
never          35095
former          9352
current         9286
not current     6447
ever            4004
Name: count, dtype: int64

In [390]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

In [391]:
t1=ColumnTransformer(transformers=[
    (("onehotencoder"),OneHotEncoder(drop="first"),["smoking_history"])

],remainder="passthrough")
t1

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(drop='first'),
                                 ['smoking_history'])])

In [392]:
t2=ColumnTransformer(transformers=[
    (("labelencoder"),OrdinalEncoder(categories=[["Female","Male","Other"]], unknown_value=-1, handle_unknown="use_encoded_value"),[0])

])
t2

ColumnTransformer(transformers=[('labelencoder',
                                 OrdinalEncoder(categories=[['Female', 'Male',
                                                             'Other']],
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 [0])])

In [393]:
x=dataset.drop(columns=["diabetes"])
x.head(2)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,Female,80.0,0,1,never,25.19,6.6,140
1,Female,54.0,0,0,No Info,27.32,6.6,80


In [394]:
y=dataset["diabetes"]
y.head()

0    0
1    0
2    0
3    0
4    0
Name: diabetes, dtype: int64

In [395]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((80000, 8), (20000, 8), (80000,), (20000,))

In [412]:
from sklearn.pipeline import Pipeline

pipe=Pipeline(steps=[
    ("t1",t1),
    ("t2",t2),
   ("decision_tree",LogisticRegression(class_weight="balanced"))
    
])
pipe

Pipeline(steps=[('t1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  ['smoking_history'])])),
                ('t2',
                 ColumnTransformer(transformers=[('labelencoder',
                                                  OrdinalEncoder(categories=[['Female',
                                                                              'Male',
                                                                              'Other']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0])])),
                ('decision_tree', LogisticRegression(class_weight='balanced'))])

In [413]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('t1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  ['smoking_history'])])),
                ('t2',
                 ColumnTransformer(transformers=[('labelencoder',
                                                  OrdinalEncoder(categories=[['Female',
                                                                              'Male',
                                                                              'Other']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [0])])),
                ('decision_tree', LogisticRegression(class_weight='balanced'))])

In [414]:
y_pred=pipe.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [415]:
import pandas as pd
a=pd.DataFrame(y_pred)
a.value_counts()

0
0    20000
Name: count, dtype: int64

In [416]:
pipe.score(x_train,y_train)*100

91.51

In [417]:
from sklearn.metrics import accuracy_score

In [418]:
accuracy_score(y_test,y_pred)

0.9146

In [419]:
dataset.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history',
       'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetes'],
      dtype='object')

In [420]:
import pandas as pd

input_data = pd.DataFrame([{
    "gender": "Male",
    "age": 67.0,
    "hypertension": 0,
    "heart_disease": 1,
    "smoking_history": "not current",
    "bmi": 27.32,
    "HbA1c_level": 6.5,
    "blood_glucose_level": 200
}])
input_data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,Male,67.0,0,1,not current,27.32,6.5,200


In [421]:
prediction = pipe.predict(input_data)
prediction

array([0], dtype=int64)

In [422]:
from collections import Counter
print(Counter(y_train))


Counter({0: 73208, 1: 6792})
